In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import polars as pl
import pandas
import numpy as np
from tqdm import tqdm
import json

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/Nhom_4/2. Thực Hành/Visualize Data/Data

/content/drive/.shortcut-targets-by-id/1DZPvcWluXIlHK-LyXls4Ej4ze8BPH2yo/Nhom_4/2. Thực Hành/Visualize Data/Data


# Calculate time user spend watching per video

In [ ]:
user_video_df = pl.read_ndjson('relations/user-video.json')
user_video_df

seq,user_id
list[struct[2]],str
"[{""V_1395633"",[{130.0,190.0,1.0,1588431144}, {220.0,250.0,1.0,1588431234}, … {655.1,692.55,1.25,1588437514}]}, {""V_1395635"",[{135.0,170.0,1.0,1588438045}]}, … {""V_6210800"",[{6.0,13.5,1.5,1601012899}]}]","""U_112"""
"[{""V_6224262"",[{286.0,506.0,2.0,1604061626}]}]","""U_150"""
"[{""V_6432959"",[{4552.1,4556.8,1.0,1598192594}]}]","""U_172"""
"[{""V_6334508"",[{4.0,109.0,1.0,1598832781}, {119.0,254.0,1.0,1598832896}]}, {""V_6334516"",[{593.0,598.0,1.0,1598834837}]}, … {""V_6254676"",[{5.0,105.0,1.0,1603849805}]}]","""U_189"""
"[{""V_1358540"",[{59.747,64.747,1.0,1582184174}, {69.795,74.795,1.0,1582184184}, … {584.751,589.751,1.0,1582186212}]}, {""V_1358542"",[{124.588,129.588,1.0,1582186455}, {194.608,199.608,1.0,1582186525}, … {579.763,584.763,1.0,1582186910}]}, … {""V_1358782"",[{350.2,395.3,1.5,1587530324}, {492.6,95.4,1.5,1587530419}]}]","""U_197"""
"[{""V_1258459"",[{186.8,206.8,2.0,1598146356}, {276.9,306.5,2.0,1598146810}]}, {""V_1258460"",[{24.8,294.7,2.0,1598146880}, {359.4,369.4,2.0,1598147047}, {379.1,498.9,2.0,1598147082}]}, … {""V_1646770"",[{2392.8,2452.8,1.0,1598240294}, {2461.7,2466.5,1.5,1598280879}]}]","""U_382"""
"[{""V_1358540"",[{4.501,598.978027,1.0,1582990570}, {4.75,19.75,1.0,1582991256}]}, {""V_1358542"",[{4.501,204.501,1.0,1582991286}, {209.502,374.501,1.0,1582991491}, {384.501,591.916992,1.0,1582991666}]}, … {""V_1358637"",[{4.001,9.001,1.0,1586081587}, {59.751,64.751,1.0,1586081796}]}]","""U_514"""
"[{""V_7449992"",[{4.388,9.388,1.0,1600752417}]}]","""U_670"""
"[{""V_1358540"",[{92.0,99.5,1.5,1580808838}, {174.0,181.5,1.5,1580808893}, … {579.0,586.5,1.5,1580809163}]}, {""V_1358542"",[{104.0,118.5,1.5,1580809274}, {164.0,171.5,1.5,1580809314}, … {471.0,478.5,1.5,1580809519}]}, … {""V_1358685"",[{33.0,38.0,1.0,1585185425}, {83.0,88.0,1.0,1585185475}, … {319.0,324.0,1.0,1585186230}]}]","""U_783"""


In [ ]:
video_id_to_ccid = pl.read_csv("./relations/video_id-ccid.txt", separator="\t", has_header=False)
video_id_to_ccid.columns = ["video_id", "ccid"]
video_id_to_ccid.head()

video_id,ccid
str,str
"""V_234845""","""0000363DB5B6E0…"
"""V_234876""","""0000363DB5B6E0…"
"""V_234907""","""0000363DB5B6E0…"
"""V_293392""","""0000363DB5B6E0…"
"""V_293445""","""0000363DB5B6E0…"


In [ ]:
id_to_ccid = dict(zip(list(video_id_to_ccid['video_id']), list(video_id_to_ccid['ccid'])))
len(id_to_ccid)

2798892

In [ ]:
video_df = pl.read_ndjson('entities/video.json')
video_df

ccid,name,start,end,text
str,str,list[f64],list[f64],list[str]
"""0001603F826A3D…","""Video""","[1.031, 7.095, … 504.247]","[4.255, 8.119, … 509.095]","[""第二个就是短助记符在生成上面。有一个规定"", ""短助记符是这样说"", … ""那么它的要求在它都会在那个命令集的定义的时候把它明确的说出来。""]"
"""0003DB14A14A53…","""Video""","[11.799, 12.581, … 365.129]","[12.581, 17.38, … 367.336]","[""大家好"", ""今天我们讨论一维系统辐射传递一般近似解"", … ""多维系统下的通用问题""]"
"""0004A5C6F07E36…","""家禽的消化系统""","[19.2, 20.84, … 491.277]","[20.83, 23.39, … 492.315]","[""各位同学 大家好"", ""我是来自中国农业大学的曹静"", … ""是胰腺""]"
"""00059EBD1371A6…","""1.2 计算机硬件系统""","[19.372, 190.522, … 626.853]","[23.14, 194.458, … 630.757]","[""大家好 本视频介绍计算机硬件系统"", ""总线就其类别来说 在计算机系统当中包括"", … ""待数据交换结束之后 再中断CPU善后""]"
"""0005D1DC01B4EF…","""什么是供应链网络""","[24.33, 25.081, … 408.031]","[25.08, 26.775, … 408.75]","[""大家好"", ""今天我们学习的知识点"", … ""谢谢大家""]"
"""0007919ED7652A…","""1.2 英语文字的起源""","[6.29, 7.31, … 504.99]","[7.26, 9.97, … 506.57]","[""同学们好"", ""今天我们开始《英汉语言对比》课程的"", … ""同学们 再见""]"
"""0007C4A5229EE3…","""7.2.3 第三课时""","[11.1, 19.5, … 1189.26]","[13.1, 22.42, … 1190.78]","[""第二节代理权"", ""授权不明的情况还是大量存在的"", … ""关于无权代理""]"
"""0008C8CC056F4E…","""Goodbye and Si…","[1.17, 3.57, … 126.12]","[3.57, 5.31, … 130.309]","[""好了 就是这样"", ""我要跟你们说再见了"", … ""好吧 那么再见 后会有期""]"
"""000948D39EA6B0…","""Video""","[13.25, 16.66, … 420.02]","[16.66, 19.49, … 421.32]","[""我觉得咱们上了一学期的课"", ""其实我一直在说清华大学"", … ""好 谢谢大家""]"


In [ ]:
def merge_non_overlapping(segment_list):
    if len(segment_list) == 0:
        return []

    non_overlapping_segments = []
    sorted_segments = sorted(segment_list, key=lambda x: x[0])
    current_segment = sorted_segments[0]
    for next_segment in sorted_segments[1:]:
        if current_segment[1] >= next_segment[0]:
            current_segment[1] = max(current_segment[1], next_segment[1])
        else:
            non_overlapping_segments.append(current_segment)
            current_segment = next_segment
    non_overlapping_segments.append(current_segment)
    return non_overlapping_segments

In [ ]:
user_vid_time_dict = {}
for i in tqdm(range(user_video_df.shape[0])):
    user_id, videos = user_video_df['user_id'][i], user_video_df['seq'][i]
    user_vid_time_dict[user_id] = {}

    for video in list(videos):
        video_id, segments = video['video_id'], video['segment']

        if video_id not in id_to_ccid:
            continue

        ccid = id_to_ccid[video_id]

        new_segments = []
        for seq in segments:
            start = seq['start_point']
            end = seq['end_point']
            new_segments.append([start, end] if start <= end else [end, start])

        if ccid not in user_vid_time_dict[user_id]:
            user_vid_time_dict[user_id][ccid] = []

        user_vid_time_dict[user_id][ccid].extend(new_segments)

    for ccid, segments in user_vid_time_dict[user_id].items():
        non_overlap_segments = merge_non_overlapping(segments)
        t = sum([seg[1] - seg[0] for seg in non_overlap_segments])
        user_vid_time_dict[user_id][ccid] = np.round(t)

100%|██████████| 310360/310360 [08:40<00:00, 595.85it/s] 


In [ ]:
with open('time_user_spend_watching_per_course.json', 'w', encoding='utf-8') as wf:
    json.dump(user_vid_time_dict, wf)

In [ ]:
import json

with open('time_user_spend_watching_per_course.json', 'r', encoding='utf-8') as rf:
    user_vid_time_dict = json.load(rf)

user_vid_time_dict

{'U_112': {'98742EFD5E62CAF59C33DC5901307461': 157.0,
  '1555F0E7D894E1859C33DC5901307461': 35.0,
  '4ACDB16216EB28399C33DC5901307461': 52.0,
  '8E136953208995509C33DC5901307461': 12.0,
  'C89DED9BB011AB959C33DC5901307461': 319.0,
  '051FF76E67C7E8AA9C33DC5901307461': 8.0},
 'U_150': {'04108DA678783BC69C33DC5901307461': 220.0},
 'U_172': {'6F07754D30740EBE9C33DC5901307461': 5.0},
 'U_189': {'30D4C6F847B23AC29C33DC5901307461': 240.0,
  '1E0920F7B41446479C33DC5901307461': 5.0,
  'F14C8543B91DC6F49C33DC5901307461': 840.0,
  '9ADA5506F90393AA9C33DC5901307461': 317.0,
  'A4FBFED2E286AEA59C33DC5901307461': 125.0,
  '9BCE83D953F64D309C33DC5901307461': 150.0,
  'FE2DD62AD00B32ED9C33DC5901307461': 65.0,
  'F379D29116C809D19C33DC5901307461': 60.0,
  '2FA9C6826965198C9C33DC5901307461': 457.0,
  '3537FDD9AB4D5E699C33DC5901307461': 175.0,
  'D21BA4443C648AB79C33DC5901307461': 626.0,
  'BE1087F35C45A4179C33DC5901307461': 630.0,
  '586FD499D1F1DB7B9C33DC5901307461': 825.0,
  '98B15FDADD612B1E9C33DC59

# Calculate time user spend per course

In [ ]:
user_df = pl.read_ndjson('entities/user.json').select('id', 'course_order')
user_df

id,course_order
str,list[i64]
"""U_22""","[682129, 2294668]"
"""U_24""","[597214, 605512, … 2229905]"
"""U_25""",[1903985]
"""U_53""","[696679, 1704639, … 1794464]"
"""U_54""","[682442, 682164, … 1906706]"
"""U_67""",[696679]
"""U_68""","[696692, 948431]"
"""U_69""","[375775, 375778, … 880774]"
"""U_90""","[676664, 707135, … 1824921]"


In [ ]:
course_df = pl.read_ndjson('entities/course.json')
course_df

id,name,field,prerequisites,about,resource
str,str,list[str],str,str,list[struct[3]]
"""C_584313""","""《资治通鉴》导读""","[""历史学"", ""中国语言文学""]","""""","""通过老师导读，同学们可深入这…","[{[""第一课 导论与三家分晋"", ""导论"", ""导论""],""V_849"",""1.1.1""}, {[""第一课 导论与三家分晋"", ""智伯的覆亡"", ""智伯的覆亡""],""V_850"",""1.2.1""}, … {[""第十五课 隋唐霸业"", null, ""第十五课 隋唐霸业--习题""],""Ex_957"",""15.8""}]"
"""C_584329""","""微积分——极限理论与一元函数…","[""应用经济学"", ""数学"", … ""理论经济学""]","""""","""本课程是理工科的一门数学基础…","[{[""序言"", ""序言"", ""序言""],""V_1350"",""1.1.1""}, {[""第一章 实数与函数"", ""第一节 实数集的界与确界"", ""实数集的界""],""V_1351"",""2.1.1""}, … {[""第八章 级数"", null, ""第八章 级数--第六节思考与练习""],""Ex_1545"",""9.9""}]"
"""C_584381""","""新闻摄影""","[""艺术学"", ""新闻传播学""]","""""","""掌握基本的摄影技能，了解图片…","[{[""第一章 绪论"", ""第一讲 引言1"", ""引言1""],""V_1800"",""1.1.1""}, {[""第一章 绪论"", ""第二讲 引言2"", ""引言2""],""V_1801"",""1.2.1""}, … {[""大作业提交"", null, ""《大作业》提交--小节""],""Ex_1926"",""20.4""}]"
"""C_597208""","""数据挖掘：理论与算法""","[""计算机科学与技术""]","""""","""最有趣的理论+最有用的算法=…","[{[""走进数据科学：博大精深，美不胜收"", ""整装待发"", ""Video""],""V_2961"",""1.1.1""}, {[""走进数据科学：博大精深，美不胜收"", ""学而不思则罔"", ""Video""],""V_2962"",""1.3.1""}, … {[""美丽数据说：阆苑仙葩，美玉无瑕"", null, ""第十一章第一节测试题""],""Ex_3104"",""11.1""}]"
"""C_597225""","""大学计算机""",[],"""""","""大学计算机课程将以计算思维为…","[{[""第1周： 基于计算机的问题求解"", ""课程介绍"", ""开篇""],""V_4596"",""1.1.1""}, {[""第1周： 基于计算机的问题求解"", ""1.0 本章导学"", ""1.0 本章导学""],""V_4597"",""1.2.1""}, … {[""第9周：算法与程序设计"", null, ""第九周测验""],""Ex_4827"",""10.12""}]"
"""C_597229""","""财务分析与决策""","[""应用经济学"", ""管理科学与工程""]","""""","""这门课程用财务语言解构企业的…","[{[""资金的运用——认识资产"", ""1.1 绪论"", ""绪论""],""V_5042"",""2.1.1""}, {[""资金的运用——认识资产"", ""1.2 认识资产负债表"", ""认识资产负债表""],""V_5043"",""2.2.1""}, … {[""期末大作业——主观题"", null, ""期末主观题""],""Ex_5144"",""12.2""}]"
"""C_597291""","""高级英语写作""",[],"""""","""本课程能够帮助学生掌握英语段…","[{[""Chapter One Paragraph Writing"", ""1.1 Parts of a Paragraph"", ""1.1 Parts of a Paragraph""],""V_8379"",""1.1.1""}, {[""Chapter One Paragraph Writing"", ""1.2 Four Steps in Writing"", ""1.2.1 Step 1- Begin with a point""],""V_8380"",""1.2.1""}, … {[""Chapter Five Research Paper Writing"", null, ""Chapter Five Research Paper Writing""],""Ex_8432"",""5.3""}]"
"""C_597307""","""大唐兴衰""","[""历史学""]","""""","""隋唐五代史是史学名著《资治通…","[{[""第一课、隋朝开基"", ""第一节 隋帝杨坚"", ""第一节 隋帝杨坚""],""V_9394"",""1.1.1""}, {[""第一课、隋朝开基"", ""第二节 杨隋代周"", ""第二节 杨隋代周""],""V_9395"",""1.2.1""}, … {[""第十二课、落日长安"", null, ""第十二课、落日长安--习题""],""Ex_9464"",""12.5""}]"
"""C_597365""","""五分钟轻松搞定职场礼仪（20…",[],"""""","""职场“礼”为先，成功的未来不…","[{[""课程介绍动画：职场“礼”为先，成功的未来不是梦"", ""课程介绍动画"", ""课程介绍动画 — 职场“礼”为先，成功的未来不是梦""],""V_15713"",""1.1.1""}, {[""第一章 塑造优雅、大方、可亲的职业形象"", ""第一节 坐姿篇"", ""本讲导学 坐姿""],""V_15714"",""2.1.1""}, … {[""第五章 应对繁杂、多变、细致的文化差异"", ""第四节 欧美篇"", ""授课视频 讲义 — 欧美各国礼仪""],""V_15800"",""6.4.1""}]"


In [ ]:
user_course_time_dict = {}
for i in tqdm(range(user_df.shape[0])):
    user_id = user_df['id'][i]
    if user_id not in user_vid_time_dict:
        continue

    user_course_time_dict[user_id] = {}

    course_ids = user_df['course_order'][i]
    for course_id in course_ids:
        course_id = f'C_{course_id}'
        # Find videos of course
        resources = course_df.filter(pl.col('id') == course_id).select('resource')
        if not len(resources):
            continue

        # Calculate time
        resources = resources['resource'][0]
        t = 0

        for resource in resources:
            resource_id = resource['resource_id']
            if resource_id.startswith('V') and resource_id in id_to_ccid:
                ccid = id_to_ccid[resource_id]
                t += user_vid_time_dict[user_id].get(ccid, 0)

        # Update dict
        if t:
            user_course_time_dict[user_id][course_id] = t

100%|██████████| 3330294/3330294 [24:55<00:00, 2226.55it/s]


In [ ]:
user_course_time_dict

{'U_112': {'C_676932': 256.0, 'C_681083': 327.0},
 'U_150': {'C_682155': 220.0},
 'U_172': {'C_1924175': 5.0},
 'U_189': {'C_747024': 75.0, 'C_922850': 4819.0, 'C_696679': 1083.0},
 'U_197': {'C_697791': 12823.0},
 'U_382': {'C_680745': 455.0, 'C_955163': 3713.0},
 'U_514': {'C_697791': 10947.0},
 'U_670': {},
 'U_783': {'C_697791': 4974.0},
 'U_796': {'C_948414': 201.0, 'C_697802': 355.0, 'C_1738993': 170.0},
 'U_817': {'C_629559': 308.0},
 'U_981': {'C_696994': 5.0, 'C_1756063': 5.0},
 'U_1071': {'C_697791': 2452.0},
 'U_1073': {'C_697018': 85.0},
 'U_1193': {'C_676932': 6285.0},
 'U_1206': {'C_854839': 5781.0},
 'U_1323': {'C_676932': 117.0},
 'U_1361': {'C_696679': 40.0,
  'C_697024': 70.0,
  'C_735364': 118.0,
  'C_934535': 15.0},
 'U_1368': {'C_1912735': 25.0},
 'U_1418': {'C_677045': 1171.0},
 'U_1547': {'C_676932': 310.0},
 'U_1663': {'C_697791': 100.0},
 'U_1691': {'C_677031': 2361.0},
 'U_1819': {'C_697018': 1022.0,
  'C_697027': 95.0,
  'C_682228': 65.0,
  'C_2203228': 289.0

In [ ]:
len(user_course_time_dict)

203580

In [ ]:
with open('time_user_spend_per_course.json', 'w', encoding='utf-8') as wf:
    json.dump(user_course_time_dict, wf)

In [ ]:
with open('time_user_spend_per_course.json', encoding='utf-8') as rf:
    user_course_time_dict = json.load(rf)

user_course_time_dict

{'U_112': {'C_676932': 256.0, 'C_681083': 327.0},
 'U_150': {'C_682155': 220.0},
 'U_172': {'C_1924175': 5.0},
 'U_189': {'C_747024': 75.0, 'C_922850': 4819.0, 'C_696679': 1083.0},
 'U_197': {'C_697791': 12823.0},
 'U_382': {'C_680745': 455.0, 'C_955163': 3713.0},
 'U_514': {'C_697791': 10947.0},
 'U_670': {},
 'U_783': {'C_697791': 4974.0},
 'U_796': {'C_948414': 201.0, 'C_697802': 355.0, 'C_1738993': 170.0},
 'U_817': {'C_629559': 308.0},
 'U_981': {'C_696994': 5.0, 'C_1756063': 5.0},
 'U_1071': {'C_697791': 2452.0},
 'U_1073': {'C_697018': 85.0},
 'U_1193': {'C_676932': 6285.0},
 'U_1206': {'C_854839': 5781.0},
 'U_1323': {'C_676932': 117.0},
 'U_1361': {'C_696679': 40.0,
  'C_697024': 70.0,
  'C_735364': 118.0,
  'C_934535': 15.0},
 'U_1368': {'C_1912735': 25.0},
 'U_1418': {'C_677045': 1171.0},
 'U_1547': {'C_676932': 310.0},
 'U_1663': {'C_697791': 100.0},
 'U_1691': {'C_677031': 2361.0},
 'U_1819': {'C_697018': 1022.0,
  'C_697027': 95.0,
  'C_682228': 65.0,
  'C_2203228': 289.0

# Calculate video time of courses

In [ ]:
course_time_dict = {}
for i in tqdm(range(course_df.shape[0])):
    resources = course_df['resource'][i]
    course_id = course_df['id'][i]
    course_time = 0

    for resource in resources:
        resource_id = resource['resource_id']
        if resource_id.startswith('V') and resource_id in id_to_ccid:
            ccid = id_to_ccid[resource_id]
            video = video_df.filter(pl.col('ccid') == ccid).select('start', 'end')
            if video.shape[0] == 0:
                continue

            start_list = video['start'][0]
            end_list = video['end'][0]

            t = sum([e - s for s, e in zip(start_list, end_list)])
            course_time += t

    course_time_dict[course_id] = np.round(course_time)

100%|██████████| 3781/3781 [02:51<00:00, 22.04it/s]


In [ ]:
len(course_time_dict)

3781

In [ ]:
dict(sorted(course_time_dict.items(), key=lambda x: x[1], reverse=True))

{'C_943237': 1319587.0,
 'C_1844119': 741537.0,
 'C_697600': 739285.0,
 'C_681031': 736076.0,
 'C_680916': 410505.0,
 'C_1844124': 384328.0,
 'C_696919': 368399.0,
 'C_654551': 248857.0,
 'C_854667': 232620.0,
 'C_883345': 207153.0,
 'C_883372': 191124.0,
 'C_707083': 169097.0,
 'C_1756077': 164347.0,
 'C_770784': 145455.0,
 'C_1782008': 139209.0,
 'C_1789681': 139209.0,
 'C_707456': 138249.0,
 'C_735317': 129970.0,
 'C_584329': 124428.0,
 'C_697075': 124428.0,
 'C_756700': 120078.0,
 'C_1765604': 120078.0,
 'C_1833443': 120078.0,
 'C_674920': 117437.0,
 'C_677151': 116310.0,
 'C_758961': 116310.0,
 'C_2217528': 113258.0,
 'C_682624': 111060.0,
 'C_680812': 108937.0,
 'C_866769': 107955.0,
 'C_682271': 105789.0,
 'C_782490': 105789.0,
 'C_769293': 103576.0,
 'C_862634': 103576.0,
 'C_770738': 103576.0,
 'C_707032': 102301.0,
 'C_597314': 101908.0,
 'C_676671': 101908.0,
 'C_696655': 101908.0,
 'C_654504': 100569.0,
 'C_677146': 97942.0,
 'C_881424': 95801.0,
 'C_697087': 94125.0,
 'C_6

# Filter course user spend less than 20% of course's time

In [ ]:
match_course_dict = {}

for user_id in user_course_time_dict:
    # Consider 1 user
    courses = user_course_time_dict[user_id]
    valid_courses = []

    for course_id in courses:
        user_time = courses[course_id]
        course_time = course_time_dict.get(course_id, None)
        if course_time and user_time >= 0.05 * course_time:
            if user_id not in match_course_dict:
                match_course_dict[user_id] = []
            match_course_dict[user_id].append(course_id)

In [ ]:
max(map(len, match_course_dict.values()))

12

#Create Dataset

In [ ]:
import pandas as pd

In [ ]:
data = []

for user, courses in user_course_time_dict.items():
    for course, time in courses.items():
        if course in course_time_dict:
          data.append([user, course, time, course_time_dict[course]])
        else:
          data.append([user, course, time, 0])

df_test = pd.DataFrame(data, columns=['user', 'course', 'user_watching_time', 'total_course_time'])

In [ ]:
course_time_dict

{'C_584313': 38883.0,
 'C_584329': 124428.0,
 'C_584381': 35233.0,
 'C_597208': 49888.0,
 'C_597225': 0,
 'C_597229': 24238.0,
 'C_597291': 0,
 'C_597307': 30894.0,
 'C_597365': 0,
 'C_597367': 0,
 'C_645178': 0,
 'C_608132': 0,
 'C_678165': 0,
 'C_629513': 0,
 'C_629514': 38883.0,
 'C_629515': 19333.0,
 'C_629520': 33854.0,
 'C_629522': 17643.0,
 'C_654397': 0,
 'C_654551': 248857.0,
 'C_654504': 100569.0,
 'C_654505': 49400.0,
 'C_655852': 34426.0,
 'C_655850': 22015.0,
 'C_654554': 24086.0,
 'C_654506': 0,
 'C_629558': 0,
 'C_629503': 20178.0,
 'C_597314': 101908.0,
 'C_670597': 0,
 'C_674903': 42640.0,
 'C_674910': 0,
 'C_674912': 0,
 'C_674914': 0,
 'C_674917': 0,
 'C_674920': 117437.0,
 'C_674921': 22213.0,
 'C_674923': 0,
 'C_674926': 31386.0,
 'C_674930': 0,
 'C_674931': 0,
 'C_674937': 0,
 'C_674942': 23937.0,
 'C_674948': 19018.0,
 'C_674950': 19373.0,
 'C_674959': 0,
 'C_674962': 37675.0,
 'C_674966': 19333.0,
 'C_674968': 36730.0,
 'C_674971': 31015.0,
 'C_676637': 24238.0,

In [ ]:
df_test['total_course_time'].value_counts()

total_course_time
0.0        61801
27130.0    58556
47081.0    28385
8716.0      6735
19672.0     3066
           ...  
9849.0         1
17935.0        1
33351.0        1
26941.0        1
9586.0         1
Name: count, Length: 1087, dtype: int64

In [ ]:
def load_txt(fname):
    lst = []
    with open(fname, 'r', encoding='utf-8') as rf:
        for line in rf.readlines():
            lst.append(line.strip())

    return lst

valid_course_path = '/content/drive/MyDrive/Colab Notebooks/Nhom_4/3. Đồ Án Môn Học/Pre-process/Data/kgat-data/n_core_courses.txt'
valid_user_path = '/content/drive/MyDrive/Colab Notebooks/Nhom_4/3. Đồ Án Môn Học/Pre-process/Data/kgat-data/n_core_users.txt'

valid_users = load_txt(valid_user_path)
valid_courses = load_txt(valid_course_path)

In [ ]:
valid_users[:10], valid_courses[:10]

(['U_24',
  'U_105',
  'U_112',
  'U_172',
  'U_189',
  'U_191',
  'U_192',
  'U_198',
  'U_514',
  'U_670'],
 ['C_597314',
  'C_682129',
  'C_674910',
  'C_707038',
  'C_682312',
  'C_696994',
  'C_707360',
  'C_758201',
  'C_681088',
  'C_758221'])

In [ ]:
len(user_course_time_dict)

203580

In [ ]:
users_in_dict = set(user_course_time_dict.keys())
len(set(valid_users).intersection(users_in_dict))

33781

In [ ]:
courses_in_dict = set()
for user, courses in user_course_time_dict.items():
    for course_id, time in courses.items():
        courses_in_dict.add(course_id)
        if (course_time_dict[course_id] == 0):
            print(course_id, time)

len(set(valid_courses).intersection(courses_in_dict))

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
C_948468 401.0
C_2221558 3647.0
C_927963 385.0
C_948124 2077.0
C_1776290 429.0
C_948286 19.0
C_676885 10.0
C_676676 895.0
C_1434593 15.0
C_1779421 1820.0
C_1987701 971.0
C_1987701 1711.0
C_1779421 407.0
C_1987701 1276.0
C_1987701 1269.0
C_882596 63.0
C_735448 15.0
C_696904 258.0
C_1755912 97.0
C_747047 1286.0
C_697543 257.0
C_2066096 15.0
C_882596 5.0
C_2179752 9186.0
C_696899 622.0
C_682397 1209.0
C_682558 65.0
C_682696 663.0
C_1750791 1222.0
C_948399 836.0
C_1903980 185.0
C_866182 65.0
C_1903980 210.0
C_1903980 11.0
C_1903980 256.0
C_948423 606.0
C_948286 1359.0
C_735241 335.0
C_681414 40.0
C_746973 10.0
C_948399 869.0
C_682222 47.0
C_680716 1086.0
C_2097435 30.0
C_734059 80.0
C_1941101 25.0
C_676635 130.0
C_2221558 523.0
C_947161 1648.0
C_948078 30.0
C_947161 39.0
C_735241 1173.0
C_948406 40.0
C_907964 1488.0
C_1797897 768.0
C_1960452 205.0
C_947161 1097.0
C_682210 679.0
C_682222 1470.0
C_1779421 155.0
C_947161 936.0
C_676676 

2595

In [ ]:
user_course_time_dict

{'U_112': {'C_676932': 256.0, 'C_681083': 327.0},
 'U_150': {'C_682155': 220.0},
 'U_172': {'C_1924175': 5.0},
 'U_189': {'C_747024': 75.0, 'C_922850': 4819.0, 'C_696679': 1083.0},
 'U_197': {'C_697791': 12823.0},
 'U_382': {'C_680745': 455.0, 'C_955163': 3713.0},
 'U_514': {'C_697791': 10947.0},
 'U_670': {},
 'U_783': {'C_697791': 4974.0},
 'U_796': {'C_948414': 201.0, 'C_697802': 355.0, 'C_1738993': 170.0},
 'U_817': {'C_629559': 308.0},
 'U_981': {'C_696994': 5.0, 'C_1756063': 5.0},
 'U_1071': {'C_697791': 2452.0},
 'U_1073': {'C_697018': 85.0},
 'U_1193': {'C_676932': 6285.0},
 'U_1206': {'C_854839': 5781.0},
 'U_1323': {'C_676932': 117.0},
 'U_1361': {'C_696679': 40.0,
  'C_697024': 70.0,
  'C_735364': 118.0,
  'C_934535': 15.0},
 'U_1368': {'C_1912735': 25.0},
 'U_1418': {'C_677045': 1171.0},
 'U_1547': {'C_676932': 310.0},
 'U_1663': {'C_697791': 100.0},
 'U_1691': {'C_677031': 2361.0},
 'U_1819': {'C_697018': 1022.0,
  'C_697027': 95.0,
  'C_682228': 65.0,
  'C_2203228': 289.0

In [ ]:
valid_videos = video_df['ccid']
valid_videos

ccid
str
"""0001603F826A3D…"
"""0003DB14A14A53…"
"""0004A5C6F07E36…"
"""00059EBD1371A6…"
"""0005D1DC01B4EF…"
"""0007919ED7652A…"
"""0007C4A5229EE3…"
"""0008C8CC056F4E…"
"""000948D39EA6B0…"


In [ ]:
invalid_videos = set()
for segs in user_video_df['seq']:
    for seg in segs:
        if seg['video_id'] in id_to_ccid and id_to_ccid[seg['video_id']] not in valid_videos:
            # print(seg['video_id'], id_to_ccid[seg['video_id']])
            invalid_videos.add(seg['video_id'])

len(invalid_videos)

In [ ]:
len(invalid_videos)

#Make data

In [ ]:
user_valid = pl.read_csv("/content/drive/MyDrive/Colab Notebooks/Nhom_4/3. Đồ Án Môn Học/Pre-process/Data/kgat-data/user_list.txt", separator=' ')
user_valid.head()

org_id,remap_id
str,i64
"""U_24""",0
"""U_105""",1
"""U_112""",2
"""U_172""",3
"""U_189""",4


In [ ]:
user_mapping = {user_valid['remap_id'][id]: user_valid['org_id'][id] for id in range(len(user_valid))}

In [ ]:
user_mapping

In [8]:
TRAIN_TEST_PATH = '/content/drive/MyDrive/Colab Notebooks/Nhom_4/3. Đồ Án Môn Học/Pre-process/Data/kgat-data/train_val_test.txt'
NEW_FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/Nhom_4/3. Đồ Án Môn Học/Pre-process/Data/kgat-data/first_10_lines.txt'

cn = 0

with open(TRAIN_TEST_PATH, 'r', encoding='utf-8') as rf:
    with open(NEW_FILE_PATH, 'w', encoding='utf-8') as wf:
        for line in rf:
            if cn < 10:
                wf.write(line)
                cn += 1
            else:
                break
